In [1]:
import numpy as np
import matplotlib.pyplot as plt
import interneuron_amb
import stellate
from scipy import signal,optimize,stats
import phase_prec_hf as hf_p
import seaborn as sns
import sim_utils as s_utils
from neuron import h

h.load_file("stdrun.hoc")
h.celsius = 37
h.cvode.active(1)

import importlib
import analysis_utils as a_utils
importlib.reload(interneuron_amb)
importlib.reload(stellate)

<module 'stellate' from '/home/govind/gov_semproject/GridCellsTheta/GridCellsCond/stellate.py'>

In [2]:
print(h.dt)

0.025


In [ ]:
#Creating neurons and specifying weights
stellate_0 = stellate.Stellate(0)
interneuron_0 = interneuron_amb.Interneuron(0)

is_wt,si_wt = 1.0,1.0
is_inhb_gmax,si_exc_gmax = 1e-3,1e-3

nc_is = h.NetCon(interneuron_0.soma(0.5)._ref_v, stellate_0.inhb_syn, sec=interneuron_0.soma)
nc_is.weight[0] = is_inhb_gmax * is_wt

nc_si = h.NetCon(stellate_0.soma(0.5)._ref_v, interneuron_0.exc_syn, sec=stellate_0.soma)
nc_si.weight[0] = si_exc_gmax * si_wt

#theta current into interneuron

theta_ic = h.IClamp(interneuron_0.soma(0.5))
theta_ic.dur = 1e9
freq = 10
osc_amp = 2e-3
baseline_amp = 5e-4


#stellate drive
stell_ic = h.IClamp(stellate_0.soma(0.5))
stell_ic.dur = 1e9



def linear_ramp_response(base_stell_inh,stell_disn,slope):
    ramp_peak =(stell_disn - base_stell_inh) / slope
    pre_ramp = np.arange(0.0,5.0,h.dt)
    ramp_period = np.arange(5.0,ramp_peak,h.dt)
    post_ramp = np.arange(ramp_peak, ramp_peak+5.0,h.dt)

    input_arr = np.concatenate((pre_ramp,ramp_period,post_ramp))
    time_arr = np.arange(0, len(input_arr)*h.dt, h.dt)

    intrnrn_arr = osc_amp * np.sin(2 * np.pi * freq * time_arr / 1000) + np.full_like(time_arr, baseline_amp)
    intrnrn_vec = h.Vector(intrnrn_arr)
    intrnrn_vec.play(theta_ic._ref_amp, h.dt)

    stell_vec = h.Vector(input_arr)
    stell_vec.play(stell_ic._ref_amp, h.dt)

    spike_times_stell = h.Vector()
    spike_times_intrnrn = h.Vector()

    nc_stell = h.NetCon(stellate_0.soma(0.5)._ref_v, None, sec=stellate_0.soma)
    nc_stell.threshold = 0
    nc_stell.record(spike_times_stell)

    nc_intrnrn = h.NetCon(interneuron_0.soma(0.5)._ref_v, None, sec=interneuron_0.soma)
    nc_intrnrn.threshold = 0
    nc_intrnrn.record(spike_times_intrnrn)

    interneuron_input = h.Vector(theta_ic._ref_amp)
    stellate_input = h.Vector(stell_ic._ref_amp)
    time = h.Vector(h._ref_t)

    h.finitialize(-65)
    h.continuerun(int(1000*len(input_arr)*h.dt))

    del intrnrn_vec, stell_vec, time, interneuron_input, stellate_input

    return spike_times_stell,spike_times_intrnrn,time


def exponential_ramp_response(base_stell_inh,stell_disn,time_const):
    ramp_peak =(stell_disn - base_stell_inh) / slope
    pre_ramp = np.arange(0.0,5.0,h.dt)
    ramp_period = np.arange(5.0,ramp_peak,h.dt)
    post_ramp = np.arange(ramp_peak, ramp_peak+5.0,h.dt)

    input_arr = np.concatenate((pre_ramp,ramp_period,post_ramp))
    time_arr = np.arange(0, len(input_arr)*h.dt, h.dt)

    intrnrn_arr = osc_amp * np.sin(2 * np.pi * freq * time_arr / 1000) + np.full_like(time_arr, baseline_amp)
    intrnrn_vec = h.Vector(intrnrn_arr)
    intrnrn_vec.play(theta_ic._ref_amp, h.dt)

    stell_vec = h.Vector(input_arr)
    stell_vec.play(stell_ic._ref_amp, h.dt)

    spike_times_stell = h.Vector()
    spike_times_intrnrn = h.Vector()

    nc_stell = h.NetCon(stellate_0.soma(0.5)._ref_v, None, sec=stellate_0.soma)
    nc_stell.threshold = 0
    nc_stell.record(spike_times_stell)

    nc_intrnrn = h.NetCon(interneuron_0.soma(0.5)._ref_v, None, sec=interneuron_0.soma)
    nc_intrnrn.threshold = 0
    nc_intrnrn.record(spike_times_intrnrn)

    interneuron_input = h.Vector(theta_ic._ref_amp)
    stellate_input = h.Vector(stell_ic._ref_amp)
    time = h.Vector(h._ref_t)

    h.finitialize(-65)
    h.continuerun(int(1000*len(input_arr)*h.dt))

    del intrnrn_vec, stell_vec, time, interneuron_input, stellate_input

    return spike_times_stell,spike_times_intrnrn,time

    




